In [25]:
import pandas as pd

anime_ratings = pd.read_csv('anime_ratings.dat', delimiter="\t")
anime_info = pd.read_csv('anime_info.dat', delimiter="\t")
anime_ratings.head()

,User_ID,Anime_ID,Feedback
0,1,1,8
1,1,3,5
2,1,5,9
3,1,6,9
4,1,7,8


In [26]:
anime_info.head()

,anime_ids,name,genre,type,episodes,rating,members
0,1723,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,82,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,296,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,127,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,137,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


# Создание матрицы предпочтений

In [27]:
# По горизонтали будут фильмы, по вертикали - пользователи, значения - оценки
user_item_matrix = anime_ratings.pivot(index = 'Anime_ID', columns = 'User_ID', values = 'Feedback')
user_item_matrix.head()

User_ID,1,2,3,4,5,6,7,8,9,10,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,5000
Anime_ID,,,,,,,,,,,,,,,,,,,,,
1,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,7.0,NaN,NaN,8.0,NaN,NaN,9.0,...,9.0,8.0,NaN,NaN,6.0,10.0,NaN,NaN,NaN,NaN
3,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,9.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Замена пропущенных значений

In [28]:
user_item_matrix.fillna(0, inplace = True)
print(user_item_matrix.shape)
user_item_matrix.head()

(7157, 4714)


User_ID,1,2,3,4,5,6,7,8,9,10,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,5000
Anime_ID,,,,,,,,,,,,,,,,,,,,,
1,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,7.0,0.0,0.0,8.0,0.0,0.0,9.0,...,9.0,8.0,0.0,0.0,6.0,10.0,0.0,0.0,0.0,0.0
3,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,9.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Чистка матрицы

In [29]:
# Вначале сгруппируем (объединим) пользователей, возьмем только столбец Feedback
# и посчитаем, сколько было оценок у каждого пользователя
users_votes = anime_ratings.groupby('User_ID')['Feedback'].agg('count')

# Сделаем то же самое, только для фильма
movies_votes = anime_ratings.groupby('Anime_ID')['Feedback'].agg('count')

# Теперь создадим фильтр (mask)
user_mask = users_votes[users_votes > 50].index
movie_mask = movies_votes[movies_votes > 10].index

# Применим фильтры и отберем фильмы с достаточным количеством оценок
user_item_matrix = user_item_matrix.loc[movie_mask,:]

# Отберём также активных пользователей
user_item_matrix = user_item_matrix.loc[:,user_mask]

print(user_item_matrix.shape)
user_item_matrix.head()

(3531, 2193)


User_ID,2,5,7,10,13,14,15,16,18,19,...,4981,4982,4985,4987,4989,4990,4991,4992,4993,4994
Anime_ID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,6.0,0.0,0.0,0.0,6.0,0.0,0.0,8.0,0.0
2,0.0,0.0,8.0,9.0,7.0,6.0,4.0,8.0,8.0,0.0,...,0.0,8.0,0.0,0.0,10.0,10.0,9.0,8.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
5,8.0,0.0,0.0,0.0,8.0,0.0,0.0,7.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0


# Преобразование разреженной матрицы

Преобразование данных матрицы в формат сжатого хранения строкой (CSR - Compressed Sparse Row матрица)

In [30]:
from scipy.sparse import csr_matrix

# атрибут values передаст функции csr_matrix только значения датафрейма
csr_data = csr_matrix(user_item_matrix.values)
print(csr_data)

  (0, 7)	8.0
  (0, 17)	7.0
  (0, 36)	7.0
  (0, 45)	6.0
  (0, 47)	7.0
  (0, 68)	8.0
  (0, 99)	8.0
  (0, 131)	8.0
  (0, 136)	10.0
  (0, 153)	6.0
  (0, 154)	6.0
  (0, 161)	6.0
  (0, 166)	5.0
  (0, 167)	6.0
  (0, 169)	7.0
  (0, 170)	8.0
  (0, 173)	8.0
  (0, 176)	7.0
  (0, 179)	7.0
  (0, 183)	7.0
  (0, 188)	8.0
  (0, 190)	7.0
  (0, 195)	9.0
  (0, 200)	7.0
  (0, 201)	7.0
  :	:
  (3528, 1842)	8.0
  (3528, 1991)	9.0
  (3529, 655)	8.0
  (3529, 746)	10.0
  (3529, 781)	7.0
  (3529, 951)	8.0
  (3529, 1195)	4.0
  (3529, 1606)	7.0
  (3529, 1607)	7.0
  (3529, 1632)	7.0
  (3529, 1656)	7.0
  (3529, 1973)	9.0
  (3530, 693)	7.0
  (3530, 741)	8.0
  (3530, 778)	8.0
  (3530, 894)	9.0
  (3530, 951)	8.0
  (3530, 1046)	7.0
  (3530, 1347)	6.0
  (3530, 1356)	8.0
  (3530, 1423)	5.0
  (3530, 1491)	9.0
  (3530, 1576)	6.0
  (3530, 1647)	7.0
  (3530, 1787)	4.0


In [31]:
user_item_matrix = user_item_matrix.rename_axis(None, axis = 1).reset_index()
user_item_matrix.head()

,Anime_ID,2,5,7,10,13,14,15,16,18,...,4981,4982,4985,4987,4989,4990,4991,4992,4993,4994
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,6.0,0.0,0.0,0.0,6.0,0.0,0.0,8.0,0.0
1,2,0.0,0.0,8.0,9.0,7.0,6.0,4.0,8.0,8.0,...,0.0,8.0,0.0,0.0,10.0,10.0,9.0,8.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
4,5,8.0,0.0,0.0,0.0,8.0,0.0,0.0,7.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0


# Создание модели машинного обучения

Классификатор создадим на основе метода k-ближайших соседей.

In [32]:
from sklearn.neighbors import NearestNeighbors

# Создадим объект класса NearestNeighbors
# metric = ‘cosine’: косинусное сходство; algorithm = ‘brute’: полный перебор;
# n_neighbors = 20: количество соседей; n_jobs = -1: вычисления на всех свободных ядрах процессора
knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors = 20, n_jobs = -1)

# обучим модель
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

# Получение рекомендаций

In [33]:
# Давать будем 10 рекоммендаций. Будет использоваться и для определения числа ближайших соседей.
recommendations = 10
search_word = 'Berserk'

Найдём фильмы с искомым словом

In [34]:
anime_search = anime_info[anime_info['name'].str.contains(search_word)]
anime_search

,anime_ids,name,genre,type,episodes,rating,members
146,1935,Berserk,"Action, Adventure, Demons, Drama, Fantasy, Hor...",TV,25,8.40,226430
192,1920,Berserk: Ougon Jidai-hen III - Kourin,"Action, Adventure, Demons, Drama, Fantasy, Hor...",Movie,1,8.33,65594
407,1919,Berserk: Ougon Jidai-hen II - Doldrey Kouryaku,"Action, Adventure, Demons, Drama, Fantasy, Hor...",Movie,1,8.09,66721
637,1917,Berserk: Ougon Jidai-hen I - Haou no Tamago,"Action, Adventure, Demons, Fantasy, Military, ...",Movie,1,7.91,77103
3794,2005,Berserk (2016),"Action, Adventure, Demons, Drama, Fantasy, Hor...",TV,12,6.81,90817


In [37]:
anime_id = anime_search.iloc[0]['anime_ids']

# далее по индексу фильма в датасете movies найдем соответствующий индекс
# в матрице предпочтений
anime_id = user_item_matrix[user_item_matrix['Anime_ID'] == anime_id].index[0]
anime_id

1814

In [38]:
# Получим оценки этого фильма из сжатой матрицы
print(csr_data[anime_id])

  (0, 23)	7.0
  (0, 32)	7.0
  (0, 37)	5.0
  (0, 38)	8.0
  (0, 45)	10.0
  (0, 59)	9.0
  (0, 90)	8.0
  (0, 93)	8.0
  (0, 98)	7.0
  (0, 105)	9.0
  (0, 109)	7.0
  (0, 113)	10.0
  (0, 114)	7.0
  (0, 117)	10.0
  (0, 130)	8.0
  (0, 138)	8.0
  (0, 140)	10.0
  (0, 141)	10.0
  (0, 148)	10.0
  (0, 153)	8.0
  (0, 156)	9.0
  (0, 162)	8.0
  (0, 169)	8.0
  (0, 174)	5.0
  (0, 177)	9.0
  :	:
  (0, 2036)	8.0
  (0, 2041)	7.0
  (0, 2043)	9.0
  (0, 2050)	8.0
  (0, 2051)	8.0
  (0, 2058)	9.0
  (0, 2060)	9.0
  (0, 2069)	8.0
  (0, 2070)	9.0
  (0, 2083)	8.0
  (0, 2090)	7.0
  (0, 2115)	9.0
  (0, 2127)	10.0
  (0, 2129)	7.0
  (0, 2131)	8.0
  (0, 2146)	10.0
  (0, 2151)	10.0
  (0, 2156)	9.0
  (0, 2159)	10.0
  (0, 2165)	9.0
  (0, 2178)	7.0
  (0, 2181)	9.0
  (0, 2182)	6.0
  (0, 2184)	6.0
  (0, 2187)	10.0


In [39]:
# Поиск индексов ближайших соседей
# С помощью нашего классификатора найдём массив индексов фильмов (indices) и массив расстояний (distances) до них
distances, indices = knn.kneighbors(csr_data[anime_id], n_neighbors = recommendations + 1)

In [40]:
# Уберем лишние измерения через squeeze() и преобразуем массивы в списки с помощью tolist()
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()

# Упакуем индексы и расстояния в набор кортежей (tuple), хранящих пары (индекс, расстояние)
indices_distances = list(zip(indices_list, distances_list))


print(type(indices_distances[0]))

# и посмотрим на первые три пары/кортежа
print(indices_distances[:3])

<class 'tuple'>
[(1814, 3.552713678800501e-15), (1797, 0.5244994796765631), (1800, 0.5355438347554)]


In [41]:
# Отсортируем кортежи по расстоянию
indices_distances_sorted = sorted(indices_distances, key = lambda x: x[1], reverse = False)

# Надо убрать первый элемент с индексом 901 (потому что это и есть исходный фильм "Матрица")
indices_distances_sorted = indices_distances_sorted[1:]
indices_distances_sorted

[(1797, 0.5244994796765631),
 (1800, 0.5355438347554),
 (1799, 0.5405811584679839),
 (52, 0.5525889393065528),
 (11, 0.5720345316468245),
 (356, 0.5773467241199675),
 (531, 0.5810124036189368),
 (358, 0.5869107001493483),
 (382, 0.5931206643812199),
 (529, 0.5959064939803749)]

Фактически, мы уже получили рекомендации (см. выше), но надо заменить индексы фильмов их названиями.

In [43]:
# Сопоставление индексам названия фильмов
recom_list = []

# Теперь в цикле будем поочередно проходить по кортежам
for ind_dist in indices_distances_sorted:

    # искать movieId в матрице предпочтений
    matrix_movie_id = user_item_matrix.iloc[ind_dist[0]]['Anime_ID']

    # выяснять индекс этого фильма в датафрейме movies
    id = anime_info[anime_info['anime_ids'] == matrix_movie_id].index

    # брать название фильма и расстояние до него
    title = anime_info.iloc[id]['name'].values[0]
    dist = ind_dist[1]

    # помещать каждую пару в питоновский словарь
    # который, в свою очередь, станет элементом списка recom_list
    recom_list.append({'Title' : title, 'Distance' : dist})

In [44]:
recom_list[:5]

[{'Title': 'Berserk: Ougon Jidai-hen I - Haou no Tamago',
  'Distance': 0.5244994796765631},
 {'Title': 'Berserk: Ougon Jidai-hen III - Kourin',
  'Distance': 0.5355438347554},
 {'Title': 'Berserk: Ougon Jidai-hen II - Doldrey Kouryaku',
  'Distance': 0.5405811584679839},
 {'Title': 'Claymore', 'Distance': 0.5525889393065528},
 {'Title': 'Cowboy Bebop', 'Distance': 0.5720345316468245}]

In [45]:
# Для удобства выполним преобразование наших рекомендаций в датафрейм
recom_df = pd.DataFrame(recom_list, index = range(1, recommendations + 1))
recom_df

,Title,Distance
1,Berserk: Ougon Jidai-hen I - Haou no Tamago,0.524499
2,Berserk: Ougon Jidai-hen III - Kourin,0.535544
3,Berserk: Ougon Jidai-hen II - Doldrey Kouryaku,0.540581
4,Claymore,0.552589
5,Cowboy Bebop,0.572035
6,Trigun,0.577347
7,Black Lagoon,0.581012
8,Hellsing,0.586911
9,Neon Genesis Evangelion,0.593121
10,Samurai Champloo,0.595906
